In [ ]:
#@markdown ## First check that your Runtime is in GPU mode (you can run this cell to do so)

#@markdown If not, go to `Runtime` > `Change runtime type` > `Hardware accelerator` > `GPU`

!nvidia-smi

In [ ]:
#@markdown ##Install Biom3d and import the necessary Python library
!pip3 install biom3d==0.0.31

import os
import biom3d
from biom3d.train import train
from biom3d.pred import pred
from biom3d.eval import eval

In [ ]:
#@markdown ##Mount your Google drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown ## Define input and output folders

#@markdown **Where training data will be saved?** This will include two folders: the config folder (containing configuration files) and the logs folder (containing the model folders)

biom3d_dir = '/content/gdrive/MyDrive/biom3d'#@param {type:"string"}
model_name = "unet_default" #@param {type:"string"}

#@markdown **Where are your images and masks?**
img_dir = "" #@param {type:"string"}
msk_dir = "" #@param {type:"string"}

#@markdown **How many classes are there in your annotations?**

num_classes = 1 #@param {type:"number"}

#@markdown **How long would you like to train your model?**
num_epochs = 2 #@param {type:"number"}

In [ ]:
#@markdown ## Preprocess your data before training
config_path = biom3d.preprocess.auto_config_preprocess(
    img_dir=img_dir,
    msk_dir=msk_dir,
    num_classes=num_classes,
    desc=model_name,
    config_dir=os.path.join(biom3d_dir, "configs/"),
    base_config=None,
    ct_norm=False,
    num_epochs=num_epochs,
    logs_dir=os.path.join(biom3d_dir, "logs/"),
)

In [ ]:
#@markdown ## Start the training

#@markdown If you want to use of different configuration file or use an existing one, complete the following field. **Leave this field empty if you want to use the config file obtained during the preprocessing.**

custom_config_path = "" #@param {type:"string"}

if len(custom_config_path) > 0:
  config_path = custom_config_path

print("We will the following configuration file for training:", config_path)
builder = train(config=config_path)

In [ ]:
#@markdown ## Make predictions with your trained model

#@markdown **Path to the model folder: leave it empty if you want to use the model obtained during the previous training step.** This should look like `/content/gdrive/MyDrive/biom3d/logs/20230602-162331-unet_default_fold0`.

custom_log_path = "" #@param {type:"string"}

if len(custom_log_path) > 0:
  log=custom_log_path
else:
  assert 'builder' in locals().keys(), "No existing model folder found. Please complete the `custom_log_path` field or train a model."
  log = builder.base_dir

#@markdown **Prediction input directory:**
pred_dir_in = "" #@param {type:"string"}

#@markdown **Prediction output directory:** (where the prediction masks will be stored)
pred_dir_out = ""#@param {type:"string"}

dir_out = pred(
    log=log,
    dir_in=pred_dir_in,
    dir_out=pred_dir_out
    )


In [ ]:
#@markdown ## Evaluate your model on a test set

#@markdown **Prediction output directory:** where the previous prediction masks have been stored and the number of classes in your images. Leave the default values if you want to use the previous prediction path.
new_pred_dir_out = ""#@param {type:"string"}
new_num_classes = 0 #@param {type:"number"}

if new_num_classes == 0:
  assert 'num_classes' in locals().keys(), "Number of classes equal to zero and the previous number of classes does not exist. Please provide one."
  new_num_classes = num_classes

if len(new_pred_dir_out) == 0:
  assert len(dir_out) > 0, "Prediction path seems to be empty and no previous path detected."
  new_pred_dir_out = dir_out

#@markdown **Path to test masks** The test masks must correspond to the predictions.
test_msk_dir = "" #@param {type:"string"}

eval(
    dir_lab=test_msk_dir,
    dir_out=new_pred_dir_out,
    num_classes=new_num_classes
)